## Mathematical expressions used in the Conrady-model widget

We model the measured axial chromatic defocus $\Delta z$ as a three-term Conrady series  

$$
f(\lambda)=\Delta z(\lambda)=A+\frac{B}{\lambda}+\frac{C}{\lambda^{3.5}},
\qquad
\nu=\frac{1}{\lambda}\;\Longrightarrow\;
f(\nu)=A+B\nu+C\,\nu^{3.5}.
$$


---

### 1. First derivatives (“slope” plots)

| Domain | B-term only | C-term only | Total |
|--------|-------------|-------------|-------|
| **λ-domain** | $\displaystyle \frac{d\Delta z}{d\lambda}\Big|_B=-\frac{B}{\lambda^{2}}$ | $\displaystyle \frac{d\Delta z}{d\lambda}\Big|_C=-\frac{3.5\,C}{\lambda^{4.5}}$ | $\displaystyle -\frac{B}{\lambda^{2}}-\frac{3.5\,C}{\lambda^{4.5}}$ |
| **ν-domain** | $\displaystyle \frac{d\Delta z}{d\nu}\Big|_B=B$ | $\displaystyle \frac{d\Delta z}{d\nu}\Big|_C=3.5\,C\,\nu^{2.5}$ | $\displaystyle B+3.5\,C\,\nu^{2.5}$ |

These appear in the widget as  
* **green / blue solid lines** – B-term,  
* **orange / cyan solid lines** – C-term,  
* **grey / black dashed lines** – total slope.

---

### 2. Second derivatives (“curvature” plots)

| Domain | B-term only | C-term only | Total |
|--------|-------------|-------------|-------|
| **λ-domain** | $\displaystyle \frac{d^{2}\Delta z}{d\lambda^{2}}\Big|_B=\frac{2B}{\lambda^{3}}$ | $\displaystyle \frac{d^{2}\Delta z}{d\lambda^{2}}\Big|_C=\frac{17.5\,C}{\lambda^{5.5}}$ | $\displaystyle \frac{2B}{\lambda^{3}}+\frac{17.5\,C}{\lambda^{5.5}}$ |
| **ν-domain** | B-term $\equiv 0$ (second derivative of a constant) | $\displaystyle \frac{d^{2}\Delta z}{d\nu^{2}}\Big|_C=8.75\,C\,\nu^{1.5}$ | identical to C-term |

Displayed as  
* **purple (B) / magenta (C)** in λ-curvature panel,  
* **blue solid (B = 0) / cyan (C)** in ν-curvature panel,  
with the grey or black dashed line giving the combined curvature.

---

### 3. Constant term $\mathbf{A}$

$A$ shifts the entire $\Delta z$ curve vertically and **does not enter any derivative**.  
It therefore has no trace in any of the slope or curvature panels.

---

*All wavelengths $\lambda$ are in nm, inverse wavelengths $\nu$ in nm⁻¹, and $\Delta z$ is in µm.*


In [39]:
"""Interactive Conrady-formula fitter with full B/C decomposition
===============================================================

Visualise and tweak the Conrady dispersion model
    f(λ) = A + B/λ + C/λ^3.5
against measured defocus Δz(λ).

Sliders → instant update of
• **Δz vs λ**       (main physical domain)
• **Δz vs ν = 1/λ** (linearised domain)

Diagnostics (mini-plots, no overlap)
───────────────────────────────────
  dΔz/dλ   : green  = B-term, orange = C-term, grey-dashed       = total
  d²Δz/dλ² : purple = B-term, magenta = C-term, grey-dashed      = total
  dΔz/dν   : blue-dashed = B-term, cyan   = C-term, black-dashed = total
  d²Δz/dν² : (B-term ≡ 0), cyan   = C-term, black-dashed         = total

Fixed y-ranges
──────────────
  dΔz/dλ   : −15 … +5   µm / nm
  d²Δz/dλ² :  0  … 0.30 µm / nm²
  dΔz/dν   : −2e6 … +2e6 µm
  d²Δz/dν² :  0  … 4e9  µm

Dependencies
------------
- numpy
- matplotlib
- ipywidgets (Jupyter-lab/notebook)
"""

# %% Imports -----------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, interact, Output
from IPython.display import display

# %% 1. Measured data ---------------------------------------------------------
CHL_data = np.array([
    [400, 285], [410, 190], [420, 118], [430,  63], [440,  22], [450,  -9],
    [460, -31], [470, -46], [480, -56], [490, -62], [500, -64], [510, -63],
    [520, -60], [530, -54], [540, -48], [550, -39], [560, -30], [570, -20],
    [580,  -9], [590,   3], [600,  15], [610,  28], [620,  41], [630,  54],
    [640,  68], [650,  82], [660,  96], [670, 111], [680, 125], [690, 139],
    [700, 156],
])
lam_CHL     = CHL_data[:, 0]
defocus_CHL = CHL_data[:, 1]

# %% 2. Initial coefficients via ν-space LS ----------------------------------
ν = 1 / lam_CHL
X = np.vstack([np.ones_like(ν), ν, ν**3.5]).T
A0, B0, C0 = np.linalg.lstsq(X, defocus_CHL, rcond=None)[0]

# %% 3. Conrady model & derivatives -----------------------------------------

def conrady(lam_nm, A, B, C):
    ν = 1.0 / lam_nm
    return A + B*ν + C*ν**3.5

# -------- 1st derivatives ---------------------------------------------------

def components_dλ(lam_nm, B, C):
    B_part = -B/lam_nm**2
    C_part = -3.5*C/lam_nm**4.5
    return B_part, C_part, B_part + C_part

def components_dν(ν, B, C):
    B_part = np.full_like(ν, B)
    C_part = 3.5*C*ν**2.5
    return B_part, C_part, B_part + C_part

# -------- 2nd derivatives ---------------------------------------------------

def components_d2λ(lam_nm, B, C):
    B_part = 2*B/lam_nm**3
    C_part = 17.5*C/lam_nm**5.5
    return B_part, C_part, B_part + C_part

def components_d2ν(ν, C):
    B_part = np.zeros_like(ν)        # second derivative of constant B is 0
    C_part = 8.75*C*ν**1.5
    return B_part, C_part, C_part    # total = C_part (since B_part = 0)

# %% 4. Slider factory --------------------------------------------------------

def make_slider(value, label, *, span_frac=0.5, steps=150, fmt=".2e"):
    span = span_frac * max(abs(value), 1.0)
    return FloatSlider(description=label, min=value-span, max=value+span,
                       step=span/steps, value=value, continuous_update=False,
                       readout_format=fmt, layout={"width":"400px"})

A_slider = make_slider(A0, "A  [µm]", span_frac=0.2, fmt=".2f")
B_slider = make_slider(B0, "B  [µm·nm]")
C_slider = make_slider(C0, "C  [µm·nm³·⁵]")

# %% 5. Pre-computed grids ----------------------------------------------------
lam_fine = np.linspace(400, 700, 301)
ν_fine   = 1 / lam_fine

out = Output()

# %% 6. Callback --------------------------------------------------------------

def update(A, B, C):
    with out:
        out.clear_output(wait=True)

        dz_f = conrady(lam_fine, A, B, C)

        B_dλ, C_dλ, dλ_tot   = components_dλ(lam_fine, B, C)
        B_dν, C_dν, dν_tot   = components_dν(ν_fine, B, C)
        B_d2λ, C_d2λ, d2λ_tot = components_d2λ(lam_fine, B, C)
        B_d2ν, C_d2ν, d2ν_tot = components_d2ν(ν_fine, C)

        # figure & layout ---------------------------------------------------
        fig = plt.figure(figsize=(15, 6), constrained_layout=True)
        gs  = fig.add_gridspec(2, 4, height_ratios=[4, 1.8])
        ax_λ = fig.add_subplot(gs[0, 0])
        ax_ν = fig.add_subplot(gs[0, 1], sharey=ax_λ)

        # λ domain ----------------------------------------------------------
        ax_λ.plot(lam_CHL, defocus_CHL, "ko", label="measured")
        ax_λ.plot(lam_fine, dz_f, "r-", label="Conrady fit")
        ax_λ.set(xlabel="λ  [nm]", ylabel="Δz  [µm]", title="Δz vs λ")
        ax_λ.grid(True); ax_λ.legend()

        # ν domain ----------------------------------------------------------
        ax_ν.plot(ν, defocus_CHL, "ko")
        ax_ν.plot(ν_fine, dz_f, "r-")
        ax_ν.invert_xaxis(); ax_ν.grid(True)
        ax_ν.set(xlabel="ν = 1/λ  [nm⁻¹]", title="Δz vs ν")

        for ax in (ax_λ, ax_ν):
            ax.set_ylim(-500, 1000)

        # --- mini plots row -------------------------------------------------
        # dΔz/dλ components --------------------------------------------------
        ax_dλ = fig.add_subplot(gs[1, 0])
        ax_dλ.plot(lam_fine, B_dλ, "g-", label="B term")
        ax_dλ.plot(lam_fine, C_dλ, color="orange", label="C term")
        ax_dλ.plot(lam_fine, dλ_tot, "grey", ls="--", lw=1, label="total")
        ax_dλ.set_xlim(400,700); ax_dλ.set_ylim(-20,10)
        ax_dλ.set(title="dΔz/dλ", xlabel="λ  [nm]")
        ax_dλ.grid(True, lw=0.3); ax_dλ.legend(fontsize=8)

        # d²Δz/dλ² components ----------------------------------------------
        ax_d2λ = fig.add_subplot(gs[1, 1])
        ax_d2λ.plot(lam_fine, B_d2λ, "purple", label="B term")
        ax_d2λ.plot(lam_fine, C_d2λ, "magenta", label="C term")
        ax_d2λ.plot(lam_fine, d2λ_tot, "grey", ls="--", lw=1, label="total")
        ax_d2λ.set_xlim(400,700); ax_d2λ.set_ylim(-0.1,0.3)
        ax_d2λ.set(title="d²Δz/dλ²", xlabel="λ  [nm]")
        ax_d2λ.grid(True, lw=0.3); ax_d2λ.legend(fontsize=8)

        # dΔz/dν components --------------------------------------------------
        ax_dν = fig.add_subplot(gs[1, 2])
        ax_dν.plot(ν_fine, B_dν, "b", label="B term")
        ax_dν.plot(ν_fine, C_dν, "c-", label="C term")
        ax_dν.plot(ν_fine, dν_tot, "k--", lw=1, label="total")
        ax_dν.invert_xaxis(); ax_dν.set_xlim(ν_fine.max(), ν_fine.min())
        ax_dν.set_ylim(-5_000_000, 5_000_000)
        ax_dν.set(title="dΔz/dν", xlabel="ν = 1/λ  [nm⁻¹]")
        ax_dν.grid(True, lw=0.3); ax_dν.legend(fontsize=8)

        # d²Δz/dν² components ----------------------------------------------
        ax_d2ν = fig.add_subplot(gs[1, 3])
        ax_d2ν.plot(ν_fine, B_d2ν, "b", label="B term (0)")
        ax_d2ν.plot(ν_fine, C_d2ν, "c-", label="C term")
        ax_d2ν.plot(ν_fine, d2ν_tot, "k--", lw=1, label="total")
        ax_d2ν.invert_xaxis(); ax_d2ν.set_xlim(ν_fine.max(), ν_fine.min())
        ax_d2ν.set_ylim(0,4_000_000_000)
        ax_d2ν.set(title="d²Δz/dν²", xlabel="ν = 1/λ  [nm⁻¹]")
        ax_d2ν.grid(True, lw=0.3); ax_d2ν.legend(fontsize=8)

        plt.show()

        print(f"A = {A:+.2f} µm   B = {B:+.2e} µm·nm   C = {C:+.2e} µm·nm³·⁵")

# %% 7. Launch widget ---------------------------------------------------------
interact(update, A=A_slider, B=B_slider, C=C_slider)

display(out)


interactive(children=(FloatSlider(value=2018.5574805080564, continuous_update=False, description='A  [µm]', la…

Output()